# Part 3: Early exploration of multimodal emotion recognition

Previously, we implemented the TIMNet Model on multimodal data. We now continue with exploration of the dataset, and define a base model class, upon which various models can be explored. We compare the performance of early and late fusion, as well as training using individual modalities versus multiple modalities. 

## Import Libraries

In [2]:
import os
import sys
import copy
import pickle
import datetime
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import keras.api._v2.keras as keras
import keras.backend as K
from keras.optimizers.legacy import Adam
from keras.optimizers import SGD
from keras import callbacks
from keras.models import Model
from keras.layers import Layer, Dense, Input, Activation, Lambda, BatchNormalization, Conv1D, SpatialDropout1D, add, GlobalAveragePooling1D, LSTM, Dense, concatenate, TimeDistributed
from keras.activations import sigmoid
from sklearn.model_selection import KFold
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_fscore_support, accuracy_score
import tensorflow as tf
from tensorflow.keras import optimizers

## Load data

In [3]:
# Load multimodal data

from collections import Counter
# emotion_counter = Counter()

def createOneHot(train_label, test_label):
    
    maxlen = int(max(train_label.max(), test_label.max()))

    train = np.zeros((train_label.shape[0], train_label.shape[1], maxlen + 1))
    test = np.zeros((test_label.shape[0], test_label.shape[1], maxlen + 1))

    for i in range(train_label.shape[0]):
        for j in range(train_label.shape[1]):
            train[i, j, train_label[i, j]] = 1

    for i in range(test_label.shape[0]):
        for j in range(test_label.shape[1]):
            test[i, j, test_label[i, j]] = 1

    return train, test

# Load data
f = open("dataset/iemocap/raw/IEMOCAP_features_raw.pkl", "rb")
u = pickle._Unpickler(f)
u.encoding = 'latin1'
videoIDs, videoSpeakers, videoLabels, videoText, videoAudio, videoVisual, videoSentence, trainVid, testVid = u.load()
# label index mapping = {'hap':0, 'sad':1, 'neu':2, 'ang':3, 'exc':4, 'fru':5}

print('Number of training samples: ',len(trainVid))
print('Number of testing samples: ',len(testVid))

train_audio = []
train_text = []
train_visual = []
train_seq_len = []
train_label = []

test_audio = []
test_text = []
test_visual = []
test_seq_len = []
test_label = []
for vid in trainVid:
    train_seq_len.append(len(videoIDs[vid]))
for vid in testVid:
    test_seq_len.append(len(videoIDs[vid]))

max_len = max(max(train_seq_len), max(test_seq_len))
print('Max length of sequences: ', max_len)
print()

for vid in trainVid:
    # emotion_counter.update(videoLabels[vid])
    train_label.append(videoLabels[vid] + [0] * (max_len - len(videoIDs[vid])))
    pad = [np.zeros(videoText[vid][0].shape)] * (max_len - len(videoIDs[vid]))
    text = np.stack(videoText[vid] + pad, axis=0)
    train_text.append(text)

    pad = [np.zeros(videoAudio[vid][0].shape)] * (max_len - len(videoIDs[vid]))
    audio = np.stack(videoAudio[vid] + pad, axis=0)
    train_audio.append(audio)

    pad = [np.zeros(videoVisual[vid][0].shape)] * (max_len - len(videoIDs[vid]))
    video = np.stack(videoVisual[vid] + pad, axis=0)
    train_visual.append(video)

for vid in testVid:
    test_label.append(videoLabels[vid] + [0] * (max_len - len(videoIDs[vid])))
    pad = [np.zeros(videoText[vid][0].shape)] * (max_len - len(videoIDs[vid]))
    text = np.stack(videoText[vid] + pad, axis=0)
    test_text.append(text)

    pad = [np.zeros(videoAudio[vid][0].shape)] * (max_len - len(videoIDs[vid]))
    audio = np.stack(videoAudio[vid] + pad, axis=0)
    test_audio.append(audio)

    pad = [np.zeros(videoVisual[vid][0].shape)] * (max_len - len(videoIDs[vid]))
    video = np.stack(videoVisual[vid] + pad, axis=0)
    test_visual.append(video)

train_text = np.stack(train_text, axis=0)
train_audio = np.stack(train_audio, axis=0)
train_visual = np.stack(train_visual, axis=0)
print(f"Train text shape: {train_text.shape[0]} samples, {train_text.shape[1]} timesteps, {train_text.shape[2]} features")
print(f"Train audio shape: {train_audio.shape[0]} samples, {train_audio.shape[1]} timesteps, {train_audio.shape[2]} features")
print(f"Train visual shape: {train_visual.shape[0]} samples, {train_visual.shape[1]} timesteps, {train_visual.shape[2]} features")
print()

test_text = np.stack(test_text, axis=0)
test_audio = np.stack(test_audio, axis=0)
test_visual = np.stack(test_visual, axis=0)
print(f"Test text shape: {test_text.shape[0]} samples, {test_text.shape[1]} timesteps, {test_text.shape[2]} features")
print(f"Test audio shape: {test_audio.shape[0]} samples, {test_audio.shape[1]} timesteps, {test_audio.shape[2]} features")
print(f"Test visual shape: {test_visual.shape[0]} samples, {test_visual.shape[1]} timesteps, {test_visual.shape[2]} features")
print()

train_label = np.array(train_label)
test_label = np.array(test_label)
train_seq_len = np.array(train_seq_len)
test_seq_len = np.array(test_seq_len)

train_mask = np.zeros((train_text.shape[0], train_text.shape[1]), dtype='float')
for i in range(len(train_seq_len)):
    train_mask[i, :train_seq_len[i]] = 1.0

test_mask = np.zeros((test_text.shape[0], test_text.shape[1]), dtype='float')
for i in range(len(test_seq_len)):
    test_mask[i, :test_seq_len[i]] = 1.0

train_label, test_label = createOneHot(train_label, test_label)
train_data = np.concatenate((train_audio, train_visual, train_text), axis=-1)
test_data = np.concatenate((test_audio, test_visual, test_text), axis=-1)
audio_text_train_data = np.concatenate((train_audio, train_text), axis=-1)
audio_text_test_data = np.concatenate((test_audio, test_text), axis=-1)

print(f"Train data shape: {train_data.shape[0]} samples, {train_data.shape[1]} timesteps, {train_data.shape[2]} features")
print(f"Train label shape: {train_label.shape[0]} samples, {train_label.shape[1]} timesteps, {train_label.shape[2]} classes")
print()
print(f"Test data shape: {test_data.shape[0]} samples, {test_data.shape[1]} timesteps, {test_data.shape[2]} features")
print(f"Test label shape: {test_label.shape[0]} samples, {test_label.shape[1]} timesteps, {test_label.shape[2]} classes")
print()

# for emotion, count in emotion_counter.items():
#   print(f"{emotion}: {count}")

Number of training samples:  120
Number of testing samples:  31
Max length of sequences:  110

Train text shape: 120 samples, 110 timesteps, 100 features
Train audio shape: 120 samples, 110 timesteps, 100 features
Train visual shape: 120 samples, 110 timesteps, 512 features

Test text shape: 31 samples, 110 timesteps, 100 features
Test audio shape: 31 samples, 110 timesteps, 100 features
Test visual shape: 31 samples, 110 timesteps, 512 features

Train data shape: 120 samples, 110 timesteps, 712 features
Train label shape: 120 samples, 110 timesteps, 6 classes

Test data shape: 31 samples, 110 timesteps, 712 features
Test label shape: 31 samples, 110 timesteps, 6 classes



# Define Models

In [31]:
class BaseModel:
    def __init__(self, input_shape, num_classes):
        self.input_shape = input_shape
        self.num_classes = num_classes
        self.model = self.build_model()

    def build_model(self):
        pass

    def compile(self, lr=0.001):
        self.model.compile(
            optimizer=Adam(lr=lr),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )

    def train(self, x_train, y_train, x_val, y_val, epochs=10, batch_size=32):
        return self.model.fit(
            x_train, y_train,
            validation_data=(x_val, y_val),
            epochs=epochs,
            batch_size=batch_size
        )

    def evaluate(self, x_test, y_test):
        predictions = self.model.predict(x_test)
        y_pred = np.argmax(predictions, axis=-1).flatten()
        y_true = np.argmax(y_test, axis=-1).flatten()

        accuracy = accuracy_score(y_true, y_pred)
        precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='macro')

        # Return a dictionary of metrics
        return {
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1_score': f1,
        }


In [32]:
# Define early fusion model 

class EarlyFusionModel(BaseModel):
    def build_model(self):
        inputs = Input(shape=self.input_shape)
        x = LSTM(32, return_sequences=True)(inputs)
        outputs = Dense(self.num_classes, activation='softmax')(x)
        return Model(inputs, outputs)
    
    def train_and_evaluate_on_test(self, x_train, y_train, x_test, y_test, epochs=10, batch_size=32):
        # Train the model on the entire training dataset
        history = self.model.fit(
            x_train, y_train,
            epochs=epochs,
            batch_size=batch_size,
            verbose=1
        )

        # Evaluate the model on the test dataset
        test_metrics = self.evaluate(x_test, y_test)

        return history, test_metrics

In [33]:
# Define late fusion model

class LateFusionModel(BaseModel):
    def __init__(self, input_shapes, num_classes):
        self.input_shapes = input_shapes  # A list of input shapes for each modality
        super().__init__(input_shape=None, num_classes=num_classes)

    def build_model(self):
        inputs = [Input(shape=shape) for shape in self.input_shapes]
        features = [LSTM(32, return_sequences=True)(input_tensor) for input_tensor in inputs]
        combined_features = concatenate(features, axis=-1)
        outputs = TimeDistributed(Dense(self.num_classes, activation='softmax'))(combined_features)
        model = Model(inputs=inputs, outputs=outputs)
        return model
    
    def train_and_evaluate_on_test(self):
        history = self.model.fit(
            [train_audio, train_visual, train_text], train_label,
            epochs=10,
            batch_size=32,
            verbose=1
        )
        test_metrics = self.evaluate([test_audio, test_visual, test_text], test_label)
        return history, test_metrics

# Model Training

## With all modalities

### Early fusion model

In [34]:
# Perform cross validation
def cross_val_training(model_class, data, labels, num_folds=5, epochs=10, batch_size=32):
    kf = KFold(n_splits=num_folds, shuffle=True)
    accuracies, precisions, recalls, f1_scores = [], [], [], []

    for train_index, val_index in kf.split(data):
        x_train_fold, x_val_fold = data[train_index], data[val_index]
        y_train_fold, y_val_fold = labels[train_index], labels[val_index]

        model = model_class(input_shape=x_train_fold.shape[1:], num_classes=y_train_fold.shape[2])
        model.compile(lr=0.001)
        model.train(x_train_fold, y_train_fold, x_val_fold, y_val_fold, epochs=epochs, batch_size=batch_size)
        
        # Evaluate the model on the validation fold
        y_pred = model.model.predict(x_val_fold)
        y_pred = np.argmax(y_pred, axis=-1).flatten()
        y_true = np.argmax(y_val_fold, axis=-1).flatten()
        
        # Calculate and store metrics
        accuracy = accuracy_score(y_true, y_pred)
        precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='macro')
        
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        f1_scores.append(f1)
    
    # Calculate average metrics across all folds
    avg_accuracy = np.mean(accuracies)
    avg_precision = np.mean(precisions)
    avg_recall = np.mean(recalls)
    avg_f1 = np.mean(f1_scores)

    # Calculate standard deviations
    std_accuracy = np.std(accuracies)
    std_precision = np.std(precisions)
    std_recall = np.std(recalls)
    std_f1 = np.std(f1_scores)

    print(f"K-Fold Cross-Validation Results ({num_folds} folds):")
    print(f"Average Accuracy: {avg_accuracy:.4f} ± {std_accuracy:.4f}")
    print(f"Average Precision: {avg_precision:.4f} ± {std_precision:.4f}")
    print(f"Average Recall: {avg_recall:.4f} ± {std_recall:.4f}")
    print(f"Average F1-Score: {avg_f1:.4f} ± {std_f1:.4f}")

early_fusion_model = cross_val_training(EarlyFusionModel, train_data, train_label, num_folds=5, epochs=10, batch_size=32)

Epoch 1/10


/Users/venessa/Desktop/5342project/venv/lib/python3.8/site-packages/keras/src/optimizers/legacy/adam.py:118: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
2024-03-28 18:54:50.913278: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-28 18:54:51.052515: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-28 18:54:51.305016: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


3/3 [==============================] - ETA: 0s - loss: 1.6848 - accuracy: 0.5195

2024-03-28 18:54:51.845187: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-28 18:54:51.896798: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


3/3 [==============================] - 2s 265ms/step - loss: 1.6848 - accuracy: 0.5195 - val_loss: 1.5507 - val_accuracy: 0.7068
Epoch 2/10
3/3 [==============================] - 0s 44ms/step - loss: 1.4879 - accuracy: 0.7828 - val_loss: 1.3964 - val_accuracy: 0.8239
Epoch 3/10
3/3 [==============================] - 0s 69ms/step - loss: 1.3543 - accuracy: 0.8522 - val_loss: 1.2845 - val_accuracy: 0.8636
Epoch 4/10
3/3 [==============================] - 0s 58ms/step - loss: 1.2552 - accuracy: 0.8833 - val_loss: 1.1972 - val_accuracy: 0.8875
Epoch 5/10
3/3 [==============================] - 0s 54ms/step - loss: 1.1720 - accuracy: 0.9050 - val_loss: 1.1175 - val_accuracy: 0.9027
Epoch 6/10
3/3 [==============================] - 0s 43ms/step - loss: 1.0897 - accuracy: 0.9199 - val_loss: 1.0286 - val_accuracy: 0.9167
Epoch 7/10
3/3 [==============================] - 0s 60ms/step - loss: 0.9920 - accuracy: 0.9313 - val_loss: 0.9147 - val_accuracy: 0.9258
Epoch 8/10
3/3 [=====================

...

/Users/venessa/Desktop/5342project/venv/lib/python3.8/site-packages/keras/src/optimizers/legacy/adam.py:118: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
2024-03-28 18:55:04.496600: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-28 18:55:04.594840: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-28 18:55:04.802774: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


2/3 [===================>..........] - ETA: 0s - loss: 1.8001 - accuracy: 0.2621

2024-03-28 18:55:05.318117: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-28 18:55:05.370588: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


3/3 [==============================] - 1s 260ms/step - loss: 1.7564 - accuracy: 0.3759 - val_loss: 1.6086 - val_accuracy: 0.6909
Epoch 2/10
3/3 [==============================] - 0s 69ms/step - loss: 1.5526 - accuracy: 0.7193 - val_loss: 1.4624 - val_accuracy: 0.8178
Epoch 3/10
3/3 [==============================] - 0s 70ms/step - loss: 1.4136 - accuracy: 0.8258 - val_loss: 1.3536 - val_accuracy: 0.8712
Epoch 4/10
3/3 [==============================] - 0s 64ms/step - loss: 1.3104 - accuracy: 0.8717 - val_loss: 1.2659 - val_accuracy: 0.8992
Epoch 5/10
3/3 [==============================] - 0s 81ms/step - loss: 1.2244 - accuracy: 0.8969 - val_loss: 1.1879 - val_accuracy: 0.9125
Epoch 6/10
3/3 [==============================] - 0s 48ms/step - loss: 1.1472 - accuracy: 0.9140 - val_loss: 1.1086 - val_accuracy: 0.9250
Epoch 7/10
3/3 [==============================] - 0s 48ms/step - loss: 1.0624 - accuracy: 0.9248 - val_loss: 1.0147 - val_accuracy: 0.9326
Epoch 8/10
3/3 [=====================

2024-03-28 18:55:07.103373: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-28 18:55:07.142459: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [35]:
# Train and evaluate on test data
# Instantiate the model
early_fusion_model = EarlyFusionModel(input_shape=train_data.shape[1:], num_classes=train_label.shape[2])

# Compile the model
early_fusion_model.compile(lr=0.001)

# Train and evaluate on the test dataset
history, test_metrics = early_fusion_model.train_and_evaluate_on_test(train_data, train_label, test_data, test_label, epochs=10, batch_size=32)

# Print the test metrics
print("Test Metrics:")
print(f"Accuracy: {test_metrics['accuracy']:.4f}")
print(f"Precision: {test_metrics['precision']:.4f}")
print(f"Recall: {test_metrics['recall']:.4f}")
print(f"F1 Score: {test_metrics['f1_score']:.4f}")


Epoch 1/10


/Users/venessa/Desktop/5342project/venv/lib/python3.8/site-packages/keras/src/optimizers/legacy/adam.py:118: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
2024-03-28 18:55:30.049520: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-28 18:55:30.162497: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-28 18:55:30.437001: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


4/4 [==============================] - 1s 105ms/step - loss: 1.7091 - accuracy: 0.4738
Epoch 2/10
4/4 [==============================] - 0s 49ms/step - loss: 1.4735 - accuracy: 0.8053
Epoch 3/10
4/4 [==============================] - 0s 39ms/step - loss: 1.3211 - accuracy: 0.8830
Epoch 4/10
4/4 [==============================] - 0s 45ms/step - loss: 1.2015 - accuracy: 0.9149
Epoch 5/10
4/4 [==============================] - 0s 33ms/step - loss: 1.0827 - accuracy: 0.9317
Epoch 6/10
4/4 [==============================] - 0s 33ms/step - loss: 0.9275 - accuracy: 0.9420
Epoch 7/10
4/4 [==============================] - 0s 33ms/step - loss: 0.6944 - accuracy: 0.9503
Epoch 8/10
4/4 [==============================] - 0s 30ms/step - loss: 0.4456 - accuracy: 0.9552
Epoch 9/10
4/4 [==============================] - 0s 32ms/step - loss: 0.3085 - accuracy: 0.9576
Epoch 10/10
4/4 [==============================] - 0s 32ms/step - loss: 0.2528 - accuracy: 0.9590


2024-03-28 18:55:32.544014: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-28 18:55:32.585754: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 292ms/step
Test Metrics:
Accuracy: 0.7695
Precision: 0.6460
Recall: 0.5961
F1 Score: 0.6101


## Late Fusion Model

In [12]:
def cross_val_training_late_fusion(model_class, modalities_data, labels, num_folds=5, epochs=10, batch_size=32):
    kf = KFold(n_splits=num_folds, shuffle=True)
    accuracies, precisions, recalls, f1_scores = [], [], [], []

    # Ensure modalities_data is a list of datasets, one for each modality
    if not isinstance(modalities_data, list):
        raise ValueError("modalities_data should be a list of datasets, one for each modality.")

    for train_index, val_index in kf.split(modalities_data[0]):  # Use the first modality to generate indices
        # Prepare training and validation data for each modality
        x_train_fold = [modality[train_index] for modality in modalities_data]
        x_val_fold = [modality[val_index] for modality in modalities_data]
        y_train_fold, y_val_fold = labels[train_index], labels[val_index]

        # Instantiate the model for each fold. Pass None to input_shape for LateFusionModel
        model = model_class(input_shapes=[modality.shape[1:] for modality in x_train_fold], num_classes=y_train_fold.shape[-1])
        model.compile(lr=0.001)

        # Train the model using the custom train method, which internally uses .fit()
        model.train(x_train_fold, y_train_fold, x_val_fold, y_val_fold, epochs=epochs, batch_size=batch_size)

        # Evaluate the model on the validation fold using the custom evaluate method
        metrics = model.evaluate(x_val_fold, y_val_fold)

        # Store the evaluation metrics
        accuracies.append(metrics['accuracy'])
        precisions.append(metrics['precision'])
        recalls.append(metrics['recall'])
        f1_scores.append(metrics['f1_score'])
    
    # Calculate and print the average and standard deviation of the metrics across all folds
    avg_accuracy = np.mean(accuracies)
    avg_precision = np.mean(precisions)
    avg_recall = np.mean(recalls)
    avg_f1 = np.mean(f1_scores)

    # Calculate standard deviations
    std_accuracy = np.std(accuracies)
    std_precision = np.std(precisions)
    std_recall = np.std(recalls)
    std_f1 = np.std(f1_scores)

    print(f"K-Fold Cross-Validation Results ({num_folds} folds):")
    print(f"Average Accuracy: {avg_accuracy:.4f} ± {std_accuracy:.4f}")
    print(f"Average Precision: {avg_precision:.4f} ± {std_precision:.4f}")
    print(f"Average Recall: {avg_recall:.4f} ± {std_recall:.4f}")
    print(f"Average F1-Score: {avg_f1:.4f} ± {std_f1:.4f}")

cross_val_training_late_fusion(LateFusionModel, [train_audio, train_visual, train_text], train_label, num_folds=5, epochs=10, batch_size=32)

Epoch 1/10


/Users/venessa/Desktop/5342project/venv/lib/python3.8/site-packages/keras/src/optimizers/legacy/adam.py:118: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
2024-03-28 17:41:08.837646: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-28 17:41:09.074456: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-28 17:41:09.075557: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-28 17:41:09.075654: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-28 17:41:09.332234: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-0

2/3 [===================>..........] - ETA: 0s - loss: 1.8295 - accuracy: 0.3740

2024-03-28 17:41:10.176788: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-28 17:41:10.275746: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-28 17:41:10.275773: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-28 17:41:10.276081: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


3/3 [==============================] - 2s 427ms/step - loss: 1.8126 - accuracy: 0.4455 - val_loss: 1.7328 - val_accuracy: 0.6587
Epoch 2/10
3/3 [==============================] - 0s 69ms/step - loss: 1.6938 - accuracy: 0.6411 - val_loss: 1.6403 - val_accuracy: 0.7398
Epoch 3/10
3/3 [==============================] - 0s 57ms/step - loss: 1.5873 - accuracy: 0.7341 - val_loss: 1.5487 - val_accuracy: 0.8053
Epoch 4/10
3/3 [==============================] - 0s 48ms/step - loss: 1.4838 - accuracy: 0.7897 - val_loss: 1.4519 - val_accuracy: 0.8417
Epoch 5/10
3/3 [==============================] - 0s 54ms/step - loss: 1.3729 - accuracy: 0.8370 - val_loss: 1.3350 - val_accuracy: 0.8754
Epoch 6/10
3/3 [==============================] - 0s 48ms/step - loss: 1.2438 - accuracy: 0.8652 - val_loss: 1.1779 - val_accuracy: 0.8943
Epoch 7/10
3/3 [==============================] - 0s 44ms/step - loss: 1.0712 - accuracy: 0.8803 - val_loss: 0.9490 - val_accuracy: 0.8905
Epoch 8/10
3/3 [=====================

...

3/3 [==============================] - ETA: 0s - loss: 1.7932 - accuracy: 0.3918

2024-03-28 17:41:27.361530: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-28 17:41:27.461006: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-28 17:41:27.461279: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-28 17:41:27.461395: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


3/3 [==============================] - 3s 352ms/step - loss: 1.7932 - accuracy: 0.3918 - val_loss: 1.7230 - val_accuracy: 0.6170
Epoch 2/10
3/3 [==============================] - 0s 86ms/step - loss: 1.6883 - accuracy: 0.6636 - val_loss: 1.6356 - val_accuracy: 0.7360
Epoch 3/10
3/3 [==============================] - 0s 78ms/step - loss: 1.5960 - accuracy: 0.7706 - val_loss: 1.5502 - val_accuracy: 0.8049
Epoch 4/10
3/3 [==============================] - 0s 58ms/step - loss: 1.5099 - accuracy: 0.8365 - val_loss: 1.4659 - val_accuracy: 0.8470
Epoch 5/10
3/3 [==============================] - 0s 55ms/step - loss: 1.4233 - accuracy: 0.8746 - val_loss: 1.3779 - val_accuracy: 0.8773
Epoch 6/10
3/3 [==============================] - 0s 46ms/step - loss: 1.3293 - accuracy: 0.9006 - val_loss: 1.2779 - val_accuracy: 0.8981
Epoch 7/10
3/3 [==============================] - 0s 44ms/step - loss: 1.2207 - accuracy: 0.9146 - val_loss: 1.1522 - val_accuracy: 0.9159
Epoch 8/10
3/3 [=====================

2024-03-28 17:41:29.289589: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-28 17:41:29.374317: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-28 17:41:29.374487: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-28 17:41:29.374506: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 452ms/step
K-Fold Cross-Validation Results (5 folds):
Average Accuracy: 0.8749 ± 0.0256
Average Precision: 0.8912 ± 0.0096
Average Recall: 0.7271 ± 0.0623
Average F1-Score: 0.7818 ± 0.0477


In [14]:
# Test late fusion model on test data
input_shapes = [train_audio.shape[1:], train_visual.shape[1:], train_text.shape[1:]]  # Adjust as per your data
num_classes = train_label.shape[-1]  # Assuming one-hot encoded labels

model = LateFusionModel(input_shapes=input_shapes, num_classes=num_classes)
model.compile(lr=0.001)  # Make sure your BaseModel's compile method properly configures the underlying Keras model

history, test_metrics = model.train_and_evaluate_on_test()

print("Test Metrics for Late Fusion Model:")
print(f"Accuracy: {test_metrics['accuracy']:.4f}")
print(f"Precision: {test_metrics['precision']:.4f}")
print(f"Recall: {test_metrics['recall']:.4f}")
print(f"F1 Score: {test_metrics['f1_score']:.4f}")

Epoch 1/10


/Users/venessa/Desktop/5342project/venv/lib/python3.8/site-packages/keras/src/optimizers/legacy/adam.py:118: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
2024-03-28 17:41:55.943736: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-28 17:41:56.207185: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-28 17:41:56.207207: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-28 17:41:56.207606: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-28 17:41:56.595827: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-0

4/4 [==============================] - 2s 151ms/step - loss: 1.7854 - accuracy: 0.4324
Epoch 2/10
4/4 [==============================] - 0s 71ms/step - loss: 1.6569 - accuracy: 0.7058
Epoch 3/10
4/4 [==============================] - 0s 55ms/step - loss: 1.5386 - accuracy: 0.8147
Epoch 4/10
4/4 [==============================] - 0s 56ms/step - loss: 1.4166 - accuracy: 0.8693
Epoch 5/10
4/4 [==============================] - 0s 40ms/step - loss: 1.2654 - accuracy: 0.8884
Epoch 6/10
4/4 [==============================] - 0s 49ms/step - loss: 1.0401 - accuracy: 0.8591
Epoch 7/10
4/4 [==============================] - 0s 42ms/step - loss: 0.7559 - accuracy: 0.8220
Epoch 8/10
4/4 [==============================] - 0s 48ms/step - loss: 0.5654 - accuracy: 0.8197
Epoch 9/10
4/4 [==============================] - 0s 36ms/step - loss: 0.4569 - accuracy: 0.8644
Epoch 10/10
4/4 [==============================] - 0s 40ms/step - loss: 0.3873 - accuracy: 0.9189


2024-03-28 17:41:59.392781: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-28 17:41:59.482804: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-28 17:41:59.482853: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-28 17:41:59.482905: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 483ms/step
Test Metrics for Late Fusion Model:
Accuracy: 0.7642
Precision: 0.6667
Recall: 0.5819
F1 Score: 0.5950


## With individual modalities

In [39]:
# Try with just audio data
cross_val_training(EarlyFusionModel, train_audio, train_label, num_folds=5, epochs=10, batch_size=32)

Epoch 1/10


/Users/venessa/Desktop/5342project/venv/lib/python3.8/site-packages/keras/src/optimizers/legacy/adam.py:118: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
2024-03-28 22:13:44.938961: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-28 22:13:45.036413: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-28 22:13:45.243227: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


3/3 [==============================] - ETA: 0s - loss: 1.7874 - accuracy: 0.3800

2024-03-28 22:13:45.761526: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-28 22:13:45.818776: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


3/3 [==============================] - 1s 268ms/step - loss: 1.7874 - accuracy: 0.3800 - val_loss: 1.7662 - val_accuracy: 0.5629
Epoch 2/10
3/3 [==============================] - 0s 65ms/step - loss: 1.7539 - accuracy: 0.6086 - val_loss: 1.7354 - val_accuracy: 0.6625
Epoch 3/10
3/3 [==============================] - 0s 72ms/step - loss: 1.7218 - accuracy: 0.6747 - val_loss: 1.7041 - val_accuracy: 0.6818
Epoch 4/10
3/3 [==============================] - 0s 49ms/step - loss: 1.6882 - accuracy: 0.6800 - val_loss: 1.6701 - val_accuracy: 0.6807
Epoch 5/10
3/3 [==============================] - 0s 51ms/step - loss: 1.6524 - accuracy: 0.6726 - val_loss: 1.6302 - val_accuracy: 0.6716
Epoch 6/10
3/3 [==============================] - 0s 51ms/step - loss: 1.6077 - accuracy: 0.6633 - val_loss: 1.5788 - val_accuracy: 0.6606
Epoch 7/10
3/3 [==============================] - 0s 56ms/step - loss: 1.5501 - accuracy: 0.6566 - val_loss: 1.5046 - val_accuracy: 0.6527
Epoch 8/10
3/3 [=====================

...

/Users/venessa/Desktop/5342project/venv/lib/python3.8/site-packages/keras/src/optimizers/legacy/adam.py:118: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
2024-03-28 22:13:57.390702: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-28 22:13:57.489000: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-28 22:13:57.701307: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


3/3 [==============================] - ETA: 0s - loss: 1.7771 - accuracy: 0.4352

2024-03-28 22:13:58.173504: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-28 22:13:58.224436: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


3/3 [==============================] - 1s 238ms/step - loss: 1.7771 - accuracy: 0.4352 - val_loss: 1.7566 - val_accuracy: 0.6220
Epoch 2/10
3/3 [==============================] - 0s 79ms/step - loss: 1.7447 - accuracy: 0.6093 - val_loss: 1.7339 - val_accuracy: 0.6333
Epoch 3/10
3/3 [==============================] - 0s 58ms/step - loss: 1.7124 - accuracy: 0.6275 - val_loss: 1.7098 - val_accuracy: 0.6409
Epoch 4/10
3/3 [==============================] - 0s 65ms/step - loss: 1.6786 - accuracy: 0.6374 - val_loss: 1.6814 - val_accuracy: 0.6473
Epoch 5/10
3/3 [==============================] - 0s 47ms/step - loss: 1.6391 - accuracy: 0.6417 - val_loss: 1.6439 - val_accuracy: 0.6473
Epoch 6/10
3/3 [==============================] - 0s 48ms/step - loss: 1.5870 - accuracy: 0.6416 - val_loss: 1.5846 - val_accuracy: 0.6420
Epoch 7/10
3/3 [==============================] - 0s 49ms/step - loss: 1.5108 - accuracy: 0.6402 - val_loss: 1.4758 - val_accuracy: 0.6390
Epoch 8/10
3/3 [=====================

2024-03-28 22:13:59.776665: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-28 22:13:59.815725: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
/Users/venessa/Desktop/5342project/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [40]:
cross_val_training(EarlyFusionModel, train_visual, train_label, num_folds=5, epochs=10, batch_size=32)

Epoch 1/10


/Users/venessa/Desktop/5342project/venv/lib/python3.8/site-packages/keras/src/optimizers/legacy/adam.py:118: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
2024-03-28 22:14:17.309190: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-28 22:14:17.427681: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-28 22:14:17.634722: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


2/3 [===================>..........] - ETA: 0s - loss: 1.7906 - accuracy: 0.3317

2024-03-28 22:14:18.127014: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-28 22:14:18.180002: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


3/3 [==============================] - 1s 251ms/step - loss: 1.7865 - accuracy: 0.4255 - val_loss: 1.7699 - val_accuracy: 0.5678
Epoch 2/10
3/3 [==============================] - 0s 73ms/step - loss: 1.7632 - accuracy: 0.6058 - val_loss: 1.7430 - val_accuracy: 0.5678
Epoch 3/10
3/3 [==============================] - 0s 49ms/step - loss: 1.7353 - accuracy: 0.6058 - val_loss: 1.7081 - val_accuracy: 0.5678
Epoch 4/10
3/3 [==============================] - 0s 73ms/step - loss: 1.6951 - accuracy: 0.6059 - val_loss: 1.6578 - val_accuracy: 0.5682
Epoch 5/10
3/3 [==============================] - 0s 53ms/step - loss: 1.6322 - accuracy: 0.6060 - val_loss: 1.5756 - val_accuracy: 0.5682
Epoch 6/10
3/3 [==============================] - 0s 72ms/step - loss: 1.5258 - accuracy: 0.6063 - val_loss: 1.4339 - val_accuracy: 0.5686
Epoch 7/10
3/3 [==============================] - 0s 51ms/step - loss: 1.3435 - accuracy: 0.6074 - val_loss: 1.2698 - val_accuracy: 0.5746
Epoch 8/10
3/3 [=====================

...

2024-03-28 22:14:30.990866: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-28 22:14:31.041922: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


3/3 [==============================] - 1s 241ms/step - loss: 1.7877 - accuracy: 0.4453 - val_loss: 1.7776 - val_accuracy: 0.5636
Epoch 2/10
3/3 [==============================] - 0s 81ms/step - loss: 1.7674 - accuracy: 0.6128 - val_loss: 1.7607 - val_accuracy: 0.5833
Epoch 3/10
3/3 [==============================] - 0s 57ms/step - loss: 1.7453 - accuracy: 0.6449 - val_loss: 1.7411 - val_accuracy: 0.6148
Epoch 4/10
3/3 [==============================] - 0s 83ms/step - loss: 1.7189 - accuracy: 0.6734 - val_loss: 1.7172 - val_accuracy: 0.6235
Epoch 5/10
3/3 [==============================] - 0s 65ms/step - loss: 1.6864 - accuracy: 0.6700 - val_loss: 1.6854 - val_accuracy: 0.6098
Epoch 6/10
3/3 [==============================] - 0s 54ms/step - loss: 1.6420 - accuracy: 0.6518 - val_loss: 1.6386 - val_accuracy: 0.5856
Epoch 7/10
3/3 [==============================] - 0s 59ms/step - loss: 1.5760 - accuracy: 0.6356 - val_loss: 1.5629 - val_accuracy: 0.5742
Epoch 8/10
3/3 [=====================

2024-03-28 22:14:32.724895: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-28 22:14:32.770625: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


K-Fold Cross-Validation Results (5 folds):
Average Accuracy: 0.6267 ± 0.0271
Average Precision: 0.1669 ± 0.0193
Average Recall: 0.2192 ± 0.0187
Average F1-Score: 0.1849 ± 0.0146


/Users/venessa/Desktop/5342project/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [37]:
cross_val_training(EarlyFusionModel, train_text, train_label, num_folds=5, epochs=10, batch_size=32)

Epoch 1/10


/Users/venessa/Desktop/5342project/venv/lib/python3.8/site-packages/keras/src/optimizers/legacy/adam.py:118: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
2024-03-28 22:12:00.109395: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-28 22:12:00.208092: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-28 22:12:00.509264: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


3/3 [==============================] - ETA: 0s - loss: 1.7873 - accuracy: 0.4532

2024-03-28 22:12:01.030188: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-28 22:12:01.081371: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


3/3 [==============================] - 2s 260ms/step - loss: 1.7873 - accuracy: 0.4532 - val_loss: 1.7289 - val_accuracy: 0.6019
Epoch 2/10
3/3 [==============================] - 0s 73ms/step - loss: 1.6607 - accuracy: 0.6597 - val_loss: 1.6273 - val_accuracy: 0.6970
Epoch 3/10
3/3 [==============================] - 0s 63ms/step - loss: 1.5542 - accuracy: 0.7643 - val_loss: 1.5342 - val_accuracy: 0.7697
Epoch 4/10
3/3 [==============================] - 0s 77ms/step - loss: 1.4572 - accuracy: 0.8236 - val_loss: 1.4410 - val_accuracy: 0.8223
Epoch 5/10
3/3 [==============================] - 0s 71ms/step - loss: 1.3604 - accuracy: 0.8645 - val_loss: 1.3437 - val_accuracy: 0.8610
Epoch 6/10
3/3 [==============================] - 0s 51ms/step - loss: 1.2504 - accuracy: 0.8902 - val_loss: 1.2213 - val_accuracy: 0.8773
Epoch 7/10
3/3 [==============================] - 0s 47ms/step - loss: 1.1050 - accuracy: 0.9086 - val_loss: 1.0441 - val_accuracy: 0.8943
Epoch 8/10
3/3 [=====================

...

/Users/venessa/Desktop/5342project/venv/lib/python3.8/site-packages/keras/src/optimizers/legacy/adam.py:118: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
2024-03-28 22:12:13.139613: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-28 22:12:13.237664: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-28 22:12:13.435282: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


3/3 [==============================] - ETA: 0s - loss: 1.8284 - accuracy: 0.3922

2024-03-28 22:12:13.925308: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-28 22:12:13.983926: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


3/3 [==============================] - 1s 256ms/step - loss: 1.8284 - accuracy: 0.3922 - val_loss: 1.7583 - val_accuracy: 0.5447
Epoch 2/10
3/3 [==============================] - 0s 54ms/step - loss: 1.7117 - accuracy: 0.6161 - val_loss: 1.6540 - val_accuracy: 0.6458
Epoch 3/10
3/3 [==============================] - 0s 52ms/step - loss: 1.6093 - accuracy: 0.7071 - val_loss: 1.5577 - val_accuracy: 0.7371
Epoch 4/10
3/3 [==============================] - 0s 66ms/step - loss: 1.5180 - accuracy: 0.7790 - val_loss: 1.4659 - val_accuracy: 0.7992
Epoch 5/10
3/3 [==============================] - 0s 65ms/step - loss: 1.4317 - accuracy: 0.8288 - val_loss: 1.3741 - val_accuracy: 0.8462
Epoch 6/10
3/3 [==============================] - 0s 44ms/step - loss: 1.3437 - accuracy: 0.8655 - val_loss: 1.2834 - val_accuracy: 0.8659
Epoch 7/10
3/3 [==============================] - 0s 48ms/step - loss: 1.2513 - accuracy: 0.8899 - val_loss: 1.1841 - val_accuracy: 0.8886
Epoch 8/10
3/3 [=====================

2024-03-28 22:12:15.585584: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-28 22:12:15.624311: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [38]:
# Train and evaluate based on test data for text modality
# Instantiate the model
early_fusion_model = EarlyFusionModel(input_shape=train_text.shape[1:], num_classes=train_label.shape[2])

# Compile the model
early_fusion_model.compile(lr=0.001)

# Train and evaluate on the test dataset
history, test_metrics = early_fusion_model.train_and_evaluate_on_test(train_text, train_label, test_text, test_label, epochs=10, batch_size=32)

# Print the test metrics
print("Test Metrics for text:")
print(f"Accuracy: {test_metrics['accuracy']:.4f}")
print(f"Precision: {test_metrics['precision']:.4f}")
print(f"Recall: {test_metrics['recall']:.4f}")
print(f"F1 Score: {test_metrics['f1_score']:.4f}")


Epoch 1/10


/Users/venessa/Desktop/5342project/venv/lib/python3.8/site-packages/keras/src/optimizers/legacy/adam.py:118: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
2024-03-28 22:12:29.466853: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-28 22:12:29.581551: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-28 22:12:29.855925: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


4/4 [==============================] - 1s 94ms/step - loss: 1.7571 - accuracy: 0.4501
Epoch 2/10
4/4 [==============================] - 0s 44ms/step - loss: 1.6185 - accuracy: 0.6787
Epoch 3/10
4/4 [==============================] - 0s 38ms/step - loss: 1.5030 - accuracy: 0.7642
Epoch 4/10
4/4 [==============================] - 0s 43ms/step - loss: 1.4043 - accuracy: 0.8230
Epoch 5/10
4/4 [==============================] - 0s 32ms/step - loss: 1.3086 - accuracy: 0.8661
Epoch 6/10
4/4 [==============================] - 0s 40ms/step - loss: 1.2064 - accuracy: 0.8898
Epoch 7/10
4/4 [==============================] - 0s 35ms/step - loss: 1.0669 - accuracy: 0.9071
Epoch 8/10
4/4 [==============================] - 0s 31ms/step - loss: 0.8349 - accuracy: 0.9195
Epoch 9/10
4/4 [==============================] - 0s 29ms/step - loss: 0.5933 - accuracy: 0.9296
Epoch 10/10
4/4 [==============================] - 0s 29ms/step - loss: 0.4488 - accuracy: 0.9373


2024-03-28 22:12:31.836489: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-28 22:12:31.877467: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 292ms/step
Test Metrics for text:
Accuracy: 0.7560
Precision: 0.5927
Recall: 0.6039
F1 Score: 0.5925


In [41]:
# Train and evaluate based on test data for audio modality
# Instantiate the model
early_fusion_model = EarlyFusionModel(input_shape=train_audio.shape[1:], num_classes=train_label.shape[2])

# Compile the model
early_fusion_model.compile(lr=0.001)

# Train and evaluate on the test dataset
history, test_metrics = early_fusion_model.train_and_evaluate_on_test(train_audio, train_label, test_audio, test_label, epochs=10, batch_size=32)

# Print the test metrics
print("Test Metrics for audio:")
print(f"Accuracy: {test_metrics['accuracy']:.4f}")
print(f"Precision: {test_metrics['precision']:.4f}")
print(f"Recall: {test_metrics['recall']:.4f}")
print(f"F1 Score: {test_metrics['f1_score']:.4f}")


Epoch 1/10


/Users/venessa/Desktop/5342project/venv/lib/python3.8/site-packages/keras/src/optimizers/legacy/adam.py:118: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
2024-03-28 22:14:51.050722: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-28 22:14:51.181921: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-28 22:14:51.449341: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


4/4 [==============================] - 1s 106ms/step - loss: 1.7800 - accuracy: 0.4482
Epoch 2/10
4/4 [==============================] - 0s 57ms/step - loss: 1.7455 - accuracy: 0.6237
Epoch 3/10
4/4 [==============================] - 0s 27ms/step - loss: 1.7112 - accuracy: 0.6580
Epoch 4/10
4/4 [==============================] - 0s 53ms/step - loss: 1.6715 - accuracy: 0.6761
Epoch 5/10
4/4 [==============================] - 0s 39ms/step - loss: 1.6199 - accuracy: 0.6683
Epoch 6/10
4/4 [==============================] - 0s 31ms/step - loss: 1.5425 - accuracy: 0.6539
Epoch 7/10
4/4 [==============================] - 0s 33ms/step - loss: 1.3995 - accuracy: 0.6349
Epoch 8/10
4/4 [==============================] - 0s 31ms/step - loss: 1.1487 - accuracy: 0.6284
Epoch 9/10
4/4 [==============================] - 0s 29ms/step - loss: 0.9714 - accuracy: 0.6327
Epoch 10/10
1/1 [==============================] - 0s 261ms/step
Test Metrics for audio:
Accuracy: 0.6384
Precision: 0.5215
Recall: 0.349

2024-03-28 22:14:53.453785: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-28 22:14:53.493016: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [42]:
# Train and evaluate based on test data for visual modality
# Instantiate the model
early_fusion_model = EarlyFusionModel(input_shape=train_visual.shape[1:], num_classes=train_label.shape[2])

# Compile the model
early_fusion_model.compile(lr=0.001)

# Train and evaluate on the test dataset
history, test_metrics = early_fusion_model.train_and_evaluate_on_test(train_visual, train_label, test_visual, test_label, epochs=10, batch_size=32)

# Print the test metrics
print("Test Metrics for video:")
print(f"Accuracy: {test_metrics['accuracy']:.4f}")
print(f"Precision: {test_metrics['precision']:.4f}")
print(f"Recall: {test_metrics['recall']:.4f}")
print(f"F1 Score: {test_metrics['f1_score']:.4f}")


Epoch 1/10


/Users/venessa/Desktop/5342project/venv/lib/python3.8/site-packages/keras/src/optimizers/legacy/adam.py:118: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
2024-03-28 22:15:06.600393: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-28 22:15:06.726004: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-28 22:15:06.995468: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


4/4 [==============================] - 2s 127ms/step - loss: 1.7900 - accuracy: 0.4195
Epoch 2/10
4/4 [==============================] - 0s 44ms/step - loss: 1.7677 - accuracy: 0.5991
Epoch 3/10
4/4 [==============================] - 0s 41ms/step - loss: 1.7435 - accuracy: 0.6127
Epoch 4/10
4/4 [==============================] - 0s 45ms/step - loss: 1.7118 - accuracy: 0.6209
Epoch 5/10
4/4 [==============================] - 0s 57ms/step - loss: 1.6649 - accuracy: 0.6173
Epoch 6/10
4/4 [==============================] - 0s 33ms/step - loss: 1.5762 - accuracy: 0.6118
Epoch 7/10
4/4 [==============================] - 0s 34ms/step - loss: 1.3648 - accuracy: 0.6090
Epoch 8/10
4/4 [==============================] - 0s 30ms/step - loss: 1.1102 - accuracy: 0.6140
Epoch 9/10
4/4 [==============================] - 0s 34ms/step - loss: 0.9645 - accuracy: 0.6443
Epoch 10/10
4/4 [==============================] - 0s 29ms/step - loss: 0.9391 - accuracy: 0.6646


2024-03-28 22:15:09.255608: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-28 22:15:09.295831: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 376ms/step
Test Metrics for video:
Accuracy: 0.6475
Precision: 0.1958
Recall: 0.3184
F1 Score: 0.2241


/Users/venessa/Desktop/5342project/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
